# This is the project in Optimization 1

In [ ]:
from functions import *

import numpy as np
import matplotlib.pyplot as plt
# from numpy.random import default_rng
from sklearn.metrics.pairwise import pairwise_kernels


In [ ]:
np.random.seed(42)
w = np.array([1.,1.])
b = 1.

n = 100
n_A = np.random.randint(0,n)
n_B = n-n_A
margin = 5.e-1
listA,listB = TestLinear(w,b,n_A,n_B,margin)
[plt.scatter(x[0],x[1],color="r") for x in listA]
[plt.scatter(x[0],x[1],color="b") for x in listB]
plt.show()


In [ ]:
x = np.concatenate((np.array(listA),np.array(listB)))

y = np.concatenate((np.ones(n_A), -np.ones(n_B)))


G = pairwise_kernels(x, metric = kernal_linear)


print(np.shape(G))


### Projected gradient descent algorithm

\begin{align*}
    \alpha^{(k+1)} &= \alpha^{(k)}+ d^{(k)}\\
    \text{where}&\\
    d^{(k)} & = \pi _{\Omega} \left(\alpha^{(k)}-\tau_k \nabla f(\alpha^{(k)}) \right) - \alpha^{(k)}
\end{align*}

\begin{align*}
    f(\alpha) &:= \frac{1}{2} \langle\alpha, YGY\alpha\rangle- \langle1_M,\alpha\rangle\\
    \Omega &= \{\alpha \in \mathbb{R}^M : \langle y, \alpha \rangle=0 \quad \text{and} \quad 0\leq \alpha \leq C \}
\end{align*}

is a feasible set, $\pi_{\Omega}$ denotes the projection onto $\Omega$ and $\tau_k >0$ is a suitible steplength.




In [ ]:

alpha0 = np.ones(n_A+n_B)
tau = 0.01
niter = 1000
C = 0.8
# alpha = gradient_descent(alpha0, G, y, tau, niter, C=0.8, tol=1e-1)
# print(alpha)



In [ ]:
def gradient_descent(alpha0, G, y , tau0, niter, C=100, tol = 1e-7):
    '''
    Perform the gradient descent algorithm with a projected gradient step.

    Parameters
    ----------
    alpha0 : numpy array
        Initial guess for the alpha vector.
    G : numpy array
        The Gram matrix.
    y : numpy array
        The target vector.
    tau0 : float
        Initial step length.
    niter : int
        Number of iterations.
    C : float, optional
        The penalty parameter. The default is 100.
    tol : float, optional
        Tolerance for the convergence. The default is 1e-7.

    '''

    alpha = alpha0
    Y = np.diag(y)
    #Saves the A matrix to save on computation time
    A = np.dot(Y,np.dot(G,Y))
    tau = tau0

    for i in range(niter):
        
        d_k = projection(alpha - tau * gradientf(alpha, A), y=y, Y=Y, C=C) - alpha
        
        # Check for convergence when the largest component of the gradient is smaller than the tolerance
        if np.max(np.abs(d_k)) < tol:
            print("Converged after", i, "iterations")
            return alpha
        
        if i%500 == 0:
            print("Iteration", i, ":", np.max(np.abs(d_k))) 
        
        alpha = alpha + d_k 

        #Creates the Barzilai-Borwein step length
        tau = BB_step_length((alpha-d_k), alpha, gradientf, A, taumax=1e5, taumin=1e-5)
    
    print("Did not converge after", niter, "iterations", np.max(np.abs(d_k)))
    return alpha

def projection(alpha, y, Y, C=1.0, tol=1e-6, max_iter=100, delta=1e-1):  
    """
    Project the vector alpha onto the feasible region defined by the constraints.
    Parameters
    ----------
    alpha : numpy array
        The vector to be projected.
    y : numpy array
        The target vector.
    Y : numpy array
        The diagonal matrix of the target vector.
    C : float, optional
        The penalty parameter. The default is 1.
    tol : float, optional
        The tolerance for the convergence. The default is 1e-6.
    max_iter : int, optional
        The maximum number of iterations. The default is 100.
    delta : float, optional
        The step size for the binary search. The default is 1e-3.
    Returns
    -------
    projected_alpha : numpy array
        The projected vector.
    """

    beta = alpha.copy()
    low, high = -10, 10
    

    for _ in range(max_iter):
        
        # Check if the current alpha is within the feasible region
        inner_low = np.dot(y, alpha_Lagrange(beta, low, Y, C))
        
        if inner_low  > 0:
            cond= True
            while cond:
                high = low 
                low = low - delta
                cond = np.dot(y, alpha_Lagrange(beta, low, Y, C)) < 0 and np.dot(y, alpha_Lagrange(beta, high, Y, C))>0

        inner_high = np.dot(y, alpha_Lagrange(beta, high, Y, C))
        if inner_high < 0:
            cond= True
            while cond:
                low = high
                high = high + delta
                cond = np.dot(y, alpha_Lagrange(beta, low, Y, C)) < 0 and np.dot(y, alpha_Lagrange(beta, high, Y, C))>0

        # Perform binary search to find the Lagrange multiplier
        # lambda_mid = (low + high) / 2.0

        lambda_mid = high - (high-low)* np.dot(y, alpha_Lagrange(beta, high, Y, C))/(np.dot(y, alpha_Lagrange(beta, high, Y, C))-np.dot(y, alpha_Lagrange(beta, low, Y, C)))
        # print("lambda_mid", lambda_mid)
        
        projected_alpha = alpha_Lagrange(beta, lambda_mid, Y, C)
    
        constraint_value = np.dot(y, projected_alpha)
        
        # Check if the constraint is satisfied
        if abs(constraint_value) < tol:
            return projected_alpha  
    
        if constraint_value > 0:
            high = lambda_mid
        else:
            low = lambda_mid
    print("Warning: Maximum iterations reached without convergence. lambda:", lambda_mid )
    return projected_alpha

In [ ]:
alpha = gradient_descent(alpha0, G, y, tau, niter=1000, C=0.6, tol=1e-7)
# print(alpha)

In [ ]:
alpha1 = gradient_descent(alpha0, G, y, tau, niter=100, C=100, tol=1e-7)


In [ ]:
plt.scatter(range(len(alpha)), alpha1)

In [ ]:
def w_b(alpha, y, x, C=0.8):
    
    # I_s = np.where(alpha > 0 and alpha < C)
    I_s = [i for i in range(len(alpha)) if alpha[i] > 0 and alpha[i] < C]
    
    w = np.sum(alpha[I_s]*y[I_s]*x[I_s].T, axis=1) 
    b = y[I_s[0]] - np.dot(w, x[I_s[0]])
    return w, b
    


def plot_solution(x, y, alpha, w, b):

    plt.scatter(x[:,0], x[:,1], c=y)
    plt.plot([-3, 3], [(-b - w[0] * (-3)) / w[1], (-b - w[0] * 3) / w[1]], 'k-')
    #excact solution
    plt.plot([-3, 3], [(-1 - 1 * (-3)), (-1 - 1 * 3) ], 'r--')
    plt.show()

    
w, b = w_b(alpha, y, x, C = 0.6)

plot_solution(x, y, alpha, w, b)




In [ ]:
w, b = w_b(alpha1, y, x, C=100)

plot_solution(x, y, alpha1, w, b)

In [ ]:
niter = 500
C=0.006

In [ ]:

G = pairwise_kernels(x, metric = kernal_gaussian, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)
w, b = w_b(alpha, y, x)
plot_solution(x, y, alpha, w, b)

G = pairwise_kernels(x, metric = kernal_inv_multiquadratic, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)

w, b = w_b(alpha, y, x)
plot_solution(x, y, alpha, w, b)


G = pairwise_kernels(x, metric = kernal_laplacian, sigma=1)
alpha = gradient_descent(alpha0, G, y, tau, niter, C)

w, b = w_b(alpha, y, x)
plot_solution(x, y, alpha, w, b)




In [ ]:
G = pairwise_kernels(x, metric = kernal_gaussian, sigma=0.5)
alpha = gradient_descent(alpha0, G, y, tau, niter = 100, C=0.02)


In [ ]:
w, b = w_b(alpha, y, x, C=0.02)

plot_solution(x, y, alpha, w, b)

In [ ]:
w, b = w_b(alpha, y, x, C=0.8)

plot_solution(x, y, alpha, w, b)

In [ ]:
def gradient_descent_linesearch(alpha0, G, y , tau0, niter, C=100, L = 10, tol = 1e-10):
    """"
    Gradient descent with backtracking line search
    
    Parameters
    ----------

    alpha0 : np.array
        Initial point
    G : np.array
        Kernel matrix
    y : np.array
        Labels
    tau0 : float
        Initial step size
    niter : int
        Number of iterations
    C : float
        Regularization parameter
    L : int
        Number of iterations before reference function is updated

    tol : float
        Tolerance for convergence

    Returns
    -------
    alpha : np.array
        Optimal alpha
    """
    alpha = alpha0
    Y = np.diag(y)
    A = np.dot(Y,np.dot(G,Y))
    tau = tau0

    f_ref = np.inf
    f_best = f(alpha, A)
    f_c = f_best
    ell = 0
    f_ks = np.zeros(niter)
    for i in range(niter):


        d_k = projection(alpha - tau*gradientf(alpha, A), y=y, Y=Y, C=C) - alpha

        if np.max(np.abs(d_k)) < tol:
            print("Converged after", i, "iterations")
            return alpha, f_ks
        
        if i%500 == 0:
            print("Iteration", i, ":", np.max(np.abs(d_k))) 
        
        f_k = f(alpha, A)
        f_ks[i] = f_k
        if f_k < f_best:
            f_best = f_k
            f_c = f_k
            ell = 0
        else:
            f_c = np.max([f_c, f_k])
            ell = ell + 1
        if ell == L:
            f_ref = f_c
            f_c = f_k
            ell = 0

        

        if f(alpha + d_k, A) > f_ref:
            dot1 = np.dot(d_k, np.dot(A, d_k))
            dot2 = np.dot(d_k, np.dot(A, alpha))
            dot3 = np.dot(alpha, np.dot(A, d_k))
            dot4 = np.sum(d_k)
            theta = - (0.5*dot2 + 0.5 *dot3 - dot4)/dot1
            # print("theta", theta, np.shape(alpha), np.shape(d_k))
            
        else:
            theta = 1

        alpha = alpha + theta * d_k
        
        tau = BB_step_length(alpha-theta*d_k, alpha, gradientf, A, taumax=1e5, taumin=1e-5)


    print("Did not converge after", niter, "iterations")
    
    return alpha, f_ks

In [ ]:
alpha0 = np.ones(n_A+n_B)*0.5

alpha_test1, fks = gradient_descent_linesearch(alpha0, G, y, tau, niter=5000, C=0.8, L=10, tol=1e-5)
alpha_test2 = gradient_descent(alpha0, G, y, tau, niter=5000, C=0.8, tol=1e-3)


In [ ]:
plt.scatter(range(len(fks[1:-1])), fks[1:-1])
plt.show()
plt.scatter(range(len(alpha0)),alpha0)
plt.scatter( range(len(alpha_test1)),alpha_test1)
plt.show()

In [ ]:
np.random.seed(0)
alpha0 = np.random.rand((n_A+n_B))
print("alpha0", alpha0)

In [ ]:
alpha0 = np.random.rand((n_A+n_B))

niter = 3000
alpha_line = gradient_descent_linesearch(alpha0, G, y, tau, niter, C = 0.006, L=10, tol=1e-7)
alpha_grad = gradient_descent(alpha0, G, y, tau, niter, C=0.006, tol=1e-7)



In [ ]:
alpha0 = np.ones(n_A+n_B)
niter = 3000

alpha_line1 = gradient_descent_linesearch(alpha0, G, y, tau, niter, C = 0.6, L=10, tol=1e-7)
alpha_grad1 = gradient_descent(alpha0, G, y, tau, niter, C=0.6, tol=1e-3)


In [ ]:
alpha0 = np.ones(n_A+n_B)* 0.5
niter = 5000

alpha_line2 = gradient_descent_linesearch(alpha0, G, y, tau, niter, C = 0.6, L=10, tol=1e-7)

In [ ]:
plt.scatter(range(len(alpha0)), alpha0)
plt.scatter(range(len(alpha_line[0])), alpha_line[0], label="line search")
# plt.scatter(range(len(alpha_test1)),alpha_test1, label="line search test")
plt.legend()
plt.show()
# print(alpha_line)

In [ ]:

w, b= w_b(alpha_grad, y, x)
print(w,b)
plot_solution(x, y, alpha_grad, w, b)

w, b= w_b(alpha_line[0], y, x)
print(w,b)
plot_solution(x, y, alpha_line[0], w, b)


w, b = w_b(alpha_line1[0], y, x)
print(w,b)
plot_solution(x, y, alpha_line1[0], w, b)



w, b= w_b(alpha_line2[0], y, x)
print(w,b)

plot_solution(x, y, alpha_line2[0], w, b)



In [ ]:
plt.scatter(range(len(fks[1:-1])), fks[1:-1], label="line search test")
plt.scatter(range(len(alpha_line[1])), alpha_line[1], label="line search")
plt.legend()
plt.show()

In [ ]:

def w_SVM(alpha, x, y, point_x , kernel = kernal_gaussian):

    I_s = [i for i in range(len(alpha)) if alpha[i] > 0 ]
    # print("kernel: ", kernel(x[I_s[0]], point_x))
    # print("alpha: ", len(alpha[I_s]))
    # print("y: ", y[I_s])
    # print("kernel", x[I_s])
    # print("kernel: ", np.max(kernel(x[I_s], point_x)), np.min(kernel(x[I_s], point_x)))
    w = np.sum(alpha[I_s] * y[I_s] * kernel(x[I_s], point_x)) 
    return w

def b_SVM(alpha, x, y, point_x , kernel = kernal_gaussian, C=0.6):

    I_s = [i for i in range(len(alpha)) if alpha[i] > 0 and alpha[i] < C]
    
    b = y[I_s[0]]- np.sum(alpha[I_s]*y[I_s]*kernel(x[I_s], x[I_s[0]]))
    return b

points = np.linspace(-3, 3, 100)
# print(alpha)
w = [w_SVM(alpha, x, y, p) for p in points]
b = b_SVM(alpha, x, y, points[0])

plt.scatter(points, w+b, label="w_SVM")
plt.scatter(x[:,0], x[:,1], c=y)
plt.show()
# print("w, b", w, b)

# plt.plot(alpha_line1[0], label="line search")

In [ ]:
def decision_function(alpha, x, y, point_x, kernel=kernal_gaussian):
    I_s = [i for i in range(len(alpha)) if alpha[i] > 1e-5]  # threshold to avoid tiny alphas
    return np.sum(alpha[I_s] * y[I_s] * kernel(x[I_s], point_x))
